In [1]:
%%bash
mkdir src
mkdir report
mkdir build
rm -rf sample_data/

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import networkx as nx
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# --- Cấu hình ---
# Kiểm tra xem CUDA có sẵn không và đặt thiết bị tương ứng
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Model LLM (Sử dụng Qwen-1.4-7B-Chat, bạn có thể thay đổi)
# Nếu bạn muốn dùng bản 4B, hãy thay bằng 'Qwen/Qwen-1_8B-Chat' hoặc tương tự
# LLM_MODEL_NAME = "Qwen/Qwen1.5-7B-Chat"
LLM_MODEL_NAME = "Qwen/Qwen3-1.7B" # Dùng bản nhỏ hơn để chạy nhanh hơn trên CPU/ít VRAM
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# Đường dẫn lưu trữ
FAISS_INDEX_PATH = "my_faiss_index.index"
GRAPH_PATH = "my_knowledge_graph.gml"
DOC_STORE_PATH = "doc_store.json" # Để lưu trữ text của chunk, map id với text

# --- Khởi tạo Model ---
print(f"Loading LLM model: {LLM_MODEL_NAME}")
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME, trust_remote_code=True)
# Sử dụng device_map="auto" nếu có nhiều GPU hoặc muốn Transformers tự quyết định
# Đối với một GPU hoặc CPU, chỉ định rõ ràng là tốt nhất
if DEVICE == "cuda":
    llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_NAME,
        torch_dtype="auto", # Sử dụng bfloat16 nếu GPU hỗ trợ để tiết kiệm VRAM
        device_map="auto",  # Để transformers tự phân bổ lên GPU
        trust_remote_code=True
    )
else: # CPU
     llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_NAME,
        torch_dtype=torch.float32, # CPU thường dùng float32
        trust_remote_code=True
    ).to(DEVICE)

print(f"Loading embedding model: {EMBEDDING_MODEL_NAME}")
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=DEVICE)

# --- Helper Functions ---
def get_llm_response(prompt_text, max_new_tokens=250):
    messages = [{"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt_text}]
    text = llm_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = llm_tokenizer([text], return_tensors="pt").to(DEVICE)

    generated_ids = llm_model.generate(
        model_inputs.input_ids,
        max_new_tokens=max_new_tokens,
        # pad_token_id=llm_tokenizer.eos_token_id # Quan trọng với một số model
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = llm_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response.strip()

def get_embeddings(texts):
    return embedding_model.encode(texts, convert_to_tensor=True, device=DEVICE)

# --- KAG-Builder ---
class KAGBuilder:
    def __init__(self, chunk_size=500, chunk_overlap=50):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        self.graph = nx.Graph()
        self.doc_store = {} # id_chunk -> text_chunk

    def _extract_entities_simple(self, text_chunk):
        # Đơn giản hóa: trích xuất các từ viết hoa hoặc cụm từ đáng chú ý
        # Trong thực tế, bạn sẽ dùng LLM cho việc này
        prompt = f"""
        Extract up to 3 key entities (people, organizations, locations, specific concepts) from the following text.
        Return them as a comma-separated list. If no clear entities, return "None".

        Text:
        "{text_chunk}"

        Entities:
        """
        entities_str = get_llm_response(prompt, max_new_tokens=50)
        if entities_str.lower() == "none":
            return []
        return [e.strip() for e in entities_str.split(',') if e.strip()]


    def build_from_texts(self, texts_with_sources):
        """
        texts_with_sources: list of tuples, e.g., [("content of doc1", "doc1_id"), ...]
        """
        all_chunks = []
        chunk_id_counter = 0

        for text_content, source_id in texts_with_sources:
            chunks = self.text_splitter.split_text(text_content)
            for i, chunk_text in enumerate(chunks):
                chunk_id = f"{source_id}_chunk_{i}"
                self.doc_store[chunk_id] = chunk_text
                all_chunks.append({"id": chunk_id, "text": chunk_text, "source": source_id})

                # Thêm chunk vào đồ thị
                self.graph.add_node(chunk_id, type="chunk", source=source_id, text=chunk_text[:100]+"...") # Lưu 1 phần text để debug

                # Trích xuất và thêm thực thể (đơn giản)
                entities = self._extract_entities_simple(chunk_text)
                for entity_name in entities:
                    # Chuẩn hóa tên thực thể (ví dụ: viết thường)
                    normalized_entity = entity_name.lower().strip()
                    if not self.graph.has_node(normalized_entity):
                        self.graph.add_node(normalized_entity, type="entity")
                    self.graph.add_edge(chunk_id, normalized_entity, type="mentions")
                
                chunk_id_counter += 1
                if chunk_id_counter % 10 == 0:
                    print(f"Processed {chunk_id_counter} chunks...")


        # Tạo embeddings cho tất cả các chunk
        chunk_texts_for_embedding = [chunk['text'] for chunk in all_chunks]
        chunk_ids_for_embedding = [chunk['id'] for chunk in all_chunks]

        if not chunk_texts_for_embedding:
            print("No chunks to build index from.")
            return

        print("Generating embeddings for chunks...")
        embeddings = get_embeddings(chunk_texts_for_embedding).cpu().numpy()

        # Xây dựng FAISS index
        dimension = embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatL2(dimension)
        self.faiss_index = faiss.IndexIDMap(self.faiss_index) # Để map với ID của chunk

        # Tạo một mảng các ID số cho FAISS
        # Chúng ta sẽ lưu mapping từ id số này về id string của chunk
        self.faiss_id_to_chunk_id = {i: chunk_id for i, chunk_id in enumerate(chunk_ids_for_embedding)}
        numeric_ids = [i for i in range(len(chunk_ids_for_embedding))]
        
        self.faiss_index.add_with_ids(embeddings, numeric_ids)
        print(f"FAISS index built with {self.faiss_index.ntotal} vectors.")

        # Lưu trữ
        faiss.write_index(self.faiss_index, FAISS_INDEX_PATH)
        nx.write_gml(self.graph, GRAPH_PATH)
        import json
        with open(DOC_STORE_PATH, 'w') as f:
            json.dump({"doc_store": self.doc_store, "faiss_id_map": self.faiss_id_to_chunk_id}, f)
        print("Builder process completed and artifacts saved.")


# --- KAG-Solver ---
class KAGSolver:
    def __init__(self, top_k_retrieval=3):
        self.faiss_index = faiss.read_index(FAISS_INDEX_PATH)
        self.graph = nx.read_gml(GRAPH_PATH)
        import json
        with open(DOC_STORE_PATH, 'r') as f:
            saved_data = json.load(f)
            self.doc_store = saved_data['doc_store']
            # faiss_id_map từ string (do JSON) sang int
            self.faiss_id_to_chunk_id = {int(k): v for k, v in saved_data['faiss_id_map'].items()}

        self.top_k = top_k_retrieval
        self.reasoning_prompt_template = PromptTemplate(
            input_variables=["original_query", "history", "current_sub_question", "retrieved_context"],
            template="""
            Bạn là một trợ lý AI giúp trả lời một câu hỏi phức tạp bằng cách chia nhỏ nó ra.
            Câu hỏi gốc: {original_query}

            Lịch sử suy luận (các câu hỏi phụ trước đó và câu trả lời của chúng):
            {history}

            Câu hỏi phụ hiện tại: {current_sub_question}

            Ngữ cảnh được truy xuất cho câu hỏi phụ hiện tại:
            ---
            {retrieved_context}
            ---

            Dựa trên ngữ cảnh được truy xuất và lịch sử suy luận, hãy trả lời Câu hỏi phụ hiện tại.
            Nếu ngữ cảnh không đủ, hãy nêu rõ điều đó và gợi ý những gì có thể còn thiếu.
            Câu trả lời:
            """
        )
        self.synthesis_prompt_template = PromptTemplate(
            input_variables=["original_query", "reasoning_trace"],
            template="""
            Dựa trên câu hỏi gốc sau và chuỗi suy luận từng bước,
            hãy đưa ra một câu trả lời tổng hợp đầy đủ cho câu hỏi gốc.
            Kết hợp thông tin một cách mạch lạc.

            Câu hỏi gốc: {original_query}

            Chuỗi suy luận:
            {reasoning_trace}

            Câu trả lời tổng hợp cuối cùng:
            """
        )

    def _retrieve_from_vector_db(self, query_text):
        query_embedding = get_embeddings([query_text]).cpu().numpy()
        distances, indices = self.faiss_index.search(query_embedding, self.top_k)
        
        retrieved_chunk_texts = []
        for i in range(len(indices[0])):
            faiss_numeric_id = indices[0][i]
            if faiss_numeric_id != -1: # FAISS trả về -1 nếu không đủ k kết quả
                chunk_id_str = self.faiss_id_to_chunk_id.get(faiss_numeric_id)
                if chunk_id_str and chunk_id_str in self.doc_store:
                    retrieved_chunk_texts.append(f"Chunk ID: {chunk_id_str}\nContent: {self.doc_store[chunk_id_str]}\n---")
        return "\n".join(retrieved_chunk_texts)

    def _retrieve_from_kg(self, entities):
        # Đơn giản: tìm các chunk liên quan đến thực thể
        kg_info = []
        for entity in entities:
            normalized_entity = entity.lower().strip()
            if self.graph.has_node(normalized_entity):
                kg_info.append(f"Knowledge Graph information for entity '{entity}':")
                for neighbor in self.graph.neighbors(normalized_entity):
                    if self.graph.nodes[neighbor]['type'] == 'chunk':
                        kg_info.append(f"  - Mentioned in chunk: {neighbor} (Source: {self.graph.nodes[neighbor].get('source', 'N/A')})")
                        # Bạn có thể lấy thêm text của chunk này từ self.doc_store nếu cần
        return "\n".join(kg_info)

    def _plan_steps(self, original_query):
        # Sử dụng LLM để chia câu hỏi thành các bước
        # Cho baseline, chúng ta có thể yêu cầu 3 bước
        prompt = f"""
        Phân tích câu hỏi phức tạp sau thành một chuỗi gồm 3 câu hỏi phụ đơn giản hơn.
        Mỗi câu hỏi phụ nên dựa trên câu trước đó để giúp trả lời câu hỏi gốc.
        Trả về các câu hỏi phụ dưới dạng danh sách được đánh số.

        Câu hỏi gốc: "{original_query}"

        Các câu hỏi phụ:
        1. ...
        2. ...
        3. ...
        """
        plan_str = get_llm_response(prompt, max_new_tokens=150)
        # Phân tích plan_str để lấy các câu hỏi con
        sub_questions = []
        for line in plan_str.split('\n'):
            match = re.match(r"^\d+\.\s*(.+)", line)
            if match:
                sub_questions.append(match.group(1).strip())
        
        # Nếu không phân tích được, trả về một câu hỏi mặc định
        if not sub_questions:
            return [original_query] # fallback
        return sub_questions

    def solve(self, original_query, max_steps=3):
        sub_questions = self._plan_steps(original_query)
        if not sub_questions:
            print("Could not plan steps. Answering directly (basic RAG).")
            context = self._retrieve_from_vector_db(original_query)
            # (Tùy chọn: trích xuất thực thể từ original_query và lấy thông tin KG)
            final_prompt = f"Original Query: {original_query}\nContext:\n{context}\nAnswer:"
            return get_llm_response(final_prompt)

        reasoning_history = []
        reasoning_trace_for_synthesis = ""

        for i, sub_q_text in enumerate(sub_questions):
            if i >= max_steps:
                break
            
            print(f"\n--- Step {i+1}: Sub-question: {sub_q_text} ---")

            # 1. Truy xuất từ Vector DB cho câu hỏi con hiện tại
            retrieved_chunks = self._retrieve_from_vector_db(sub_q_text)
            
            # 2. (Tùy chọn) Trích xuất thực thể từ câu hỏi con và truy xuất KG
            # (Đây là phần bạn có thể làm phức tạp hơn)
            # entity_extraction_prompt = f"Extract key entities from this question: \"{sub_q_text}\". Return as comma-separated list."
            # entities_str = get_llm_response(entity_extraction_prompt, max_new_tokens=30)
            # entities_in_sub_q = [e.strip() for e in entities_str.split(',') if e.strip()]
            # kg_context = self._retrieve_from_kg(entities_in_sub_q)
            # combined_context = f"Vector DB Chunks:\n{retrieved_chunks}\n\nKnowledge Graph Context:\n{kg_context}"
            combined_context = f"Retrieved Chunks:\n{retrieved_chunks}" # Giữ đơn giản

            # 3. Tạo prompt và gọi LLM để trả lời câu hỏi con
            current_history_str = "\n".join([f"  - Q: {item['q']}\n    A: {item['a']}" for item in reasoning_history])
            
            step_prompt_input = {
                "original_query": original_query,
                "history": current_history_str if current_history_str else "No previous steps.",
                "current_sub_question": sub_q_text,
                "retrieved_context": combined_context if combined_context else "No context retrieved."
            }
            step_prompt = self.reasoning_prompt_template.format(**step_prompt_input)
            # print(f"DEBUG: Step Prompt:\n{step_prompt}")
            
            sub_answer = get_llm_response(step_prompt, max_new_tokens=300)
            print(f"Sub-answer: {sub_answer}")

            reasoning_history.append({"q": sub_q_text, "a": sub_answer})
            reasoning_trace_for_synthesis += f"Sub-Question {i+1}: {sub_q_text}\nSub-Answer {i+1}: {sub_answer}\n\n"

        # 4. Tổng hợp câu trả lời cuối cùng
        print("\n--- Synthesizing Final Answer ---")
        synthesis_prompt_input = {
            "original_query": original_query,
            "reasoning_trace": reasoning_trace_for_synthesis
        }
        final_prompt = self.synthesis_prompt_template.format(**synthesis_prompt_input)
        # print(f"DEBUG: Synthesis Prompt:\n{final_prompt}")
        final_answer = get_llm_response(final_prompt, max_new_tokens=500)
        return final_answer

# --- Main Execution ---
if __name__ == "__main__":
    # --- Giai đoạn xây dựng (chỉ chạy một lần hoặc khi dữ liệu thay đổi) ---
    # Kiểm tra xem index đã tồn tại chưa để tránh xây dựng lại không cần thiết
    if not os.path.exists(FAISS_INDEX_PATH) or not os.path.exists(GRAPH_PATH):
        print("Building KAG artifacts...")
        builder = KAGBuilder(chunk_size=300, chunk_overlap=30) # Giảm chunk size để có nhiều chunk hơn
        
        # Ví dụ dữ liệu (thay thế bằng dữ liệu thực của bạn)
        sample_docs = [
            (
                "Điều 49. Xử lý học vụ Sau mỗi học kỳ chính, đơn vị đào tạo thực hiện xử lý học vụ. "
                "Kết quả học tập của học kỳ phụ sẽ được tính vào kết quả học tập của học kỳ chính tiếp theo. "
                "1. Cảnh báo học vụ  Đầu mỗi học kỳ, đơn vị đào tạo cảnh báo đối với những sinh viên có "
                "điểm trung bình chung học kỳ đạt từ 0,80 đến dưới 0,85 đối với học kỳ đầu của khóa học; "
                "đạt từ 1,00 đến dưới 1,10 đối với các học kỳ tiếp theo hoặc đạt từ 1,10 đến dưới 1,20 "
                "đối với 2 học kỳ liên tiếp. 2. Thôi học Sinh viên được thôi học nếu có đơn xin thôi học "
                "và được Thủ trưởng đơn vị đào tạo ra quyết định đồng ý. 3. Buộc thôi học Sau mỗi học kỳ, "
                "sinh viên bị buộc thôi học nếu thuộc một trong các trường hợp sau: a) Có điểm trung bình "
                "chung học kỳ đạt dưới 0,80 đối với học kỳ đầu của khóa học; đạt dưới 1,00 đối với các học kỳ "
                "tiếp theo hoặc đạt dưới 1,10 đối với 2 học kỳ liên tiếp; b) Có điểm trung bình chung tích lũy "
                "đạt dưới 1,20 đối với sinh viên năm thứ nhất; dưới 1,40 đối với sinh viên năm thứ hai; "
                "dưới 1,60 đối với sinh viên năm thứ ba hoặc dưới 1,80 đối với sinh viên các năm tiếp theo "
                "và cuối khóa; c) Vượt quá thời gian tối đa được phép học quy định tại khoản 2, Điều 24 "
                "của Quy chế này; d) Bị kỷ luật lần thứ hai vì lý do thi hộ hoặc nhờ người thi hộ theo "
                "quy định tại mục d, khoản 10, Điều 40 của Quy chế này hoặc bị kỷ luật ở mức xóa tên "
                "khỏi danh sách sinh viên của trường.Chậm nhất 1 tháng sau khi sinh viên có quyết định "
                "buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú",
                "doc_vnu"
            )
        ]
        builder.build_from_texts(sample_docs)
    else:
        print("KAG artifacts already exist. Skipping build phase.")

    # --- Giai đoạn giải quyết truy vấn ---
    solver = KAGSolver(top_k_retrieval=2) # Lấy ít chunk hơn cho mỗi bước
    
    # query1 = "Who are the pioneers of AI and what are their main contributions, especially regarding NLP?"
    # print(f"\nSolving Query 1: {query1}")
    # answer1 = solver.solve(query1)
    # print(f"\nFinal Answer for Query 1:\n{answer1}")

    query2 = "Cho tôi hỏi khi nào sinh viên được thôi học ? Khi nào sinh viên bị buộc thôi học? Sau bao lâu sinh viên có quyết định buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú?, trả lời bằng tiếng việt "
    print(f"\nSolving Query 2: {query2}")
    answer2 = solver.solve(query2, max_steps=3) # Giới hạn số bước cho truy vấn này
    print(f"\nFinal Answer for Query 2:\n{answer2}")

c:\Users\huyho\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Loading LLM model: Qwen/Qwen3-1.7B


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.35s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


Loading embedding model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
KAG artifacts already exist. Skipping build phase.

Solving Query 2: Sau bao lâu sinh viên có quyết định buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú?, trả lời bằng tiếng việt 


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Step 1: Sub-question: Sau bao lâu sinh viên có quyết định buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú?, trả lời bằng tiếng việt  ---
Sub-answer: <think>
Okay, let's tackle this question. The user is asking how long after a student decides to withdraw (buộc thôi học) the educational institution must notify the student to return to the place of their permanent residence. The answer needs to be in Vietnamese.

First, I check the retrieved chunks. There's a chunk from doc_vnu_chunk_5 that says "1 tháng sau khi sinh viên có quyết định buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú." So that's a direct answer: 1 month after the student decides to withdraw, the institution must notify them to return to their permanent residence.

Another chunk from doc_vnu_chunk_2 talks about the decision to withdraw and the criteria for it, but it doesn't mention the notification period. So the mai

# Naive - 151.542 GFLOP/s

In [5]:
%%writefile r"C:\Users\huyho\OneDrive\Máy tính\New folder (2)"

#include <algorithm>
#include <cassert>
#include <cstdlib>
#include <functional>
#include <iostream>
#include <tuple>

using namespace std;

template<typename T>
__global__ void matmul_kernel(const T *a, const T *b, T *c, int M, int N, int K) {
  int col = blockIdx.x * 32 + (threadIdx.x % 32);
  int row = blockIdx.y * 32 + (threadIdx.x / 32);
  if (row < M && col < N) {
    c[row * N + col] = 0;
    for (int k = 0; k < K; ++k) {
      c[row * N + col] += a[row * K + k] * b[k * N + col]; // each thread accesses () global memory 2N times
    }
  }
}

template<typename T>
__host__ void verifyResult(T *a, T *b, T *c, int M, int N, int K) {
  for (int i = 0; i < M; i++) {
    for (int j = 0; j < N; j++) {
      T sum = 0;
      for (int k = 0; k < K; k++) {
        sum += a[i * K + k] * b[k * N + j];
      }
      //printf("sum: %d, c[%d * N + %d]: %d\n", sum, i, j, c[i * N + j]);
      assert(c[i * N + j] == sum);
    }
  }
  cout << "Result is correct!\n";
}

template<typename T>
__host__ void copyFromHostToDevice(T *h_a, T *h_b, T *d_a, T *d_b, int M, int N, int K) {
  size_t a_bytes = M * K * sizeof(T);
  size_t b_bytes = K * N * sizeof(T);
  cudaError_t err = cudaMemcpy(d_a, h_a, a_bytes, cudaMemcpyHostToDevice);
  if (err !=  cudaSuccess) {
    fprintf(stderr, "Failed to copy h_a to d_a (error code %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaMemcpy(d_b, h_b, b_bytes, cudaMemcpyHostToDevice);
  if (err !=  cudaSuccess) {
    fprintf(stderr, "Failed to copy h_b to d_b (error code %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T>
__host__ void executeKernel(T *d_a, T *d_b, T *d_c, int M, int N, int K) {
  // block size is the multiple of 32
  int block_dim = 32;
  dim3 block(block_dim * block_dim);
  dim3 grid((M + block_dim - 1) / block_dim, (N + block_dim - 1) / block_dim);
  matmul_kernel<T><<<grid, block>>>(d_a, d_b, d_c, M, N, K);
  cudaDeviceSynchronize();

  cudaError_t err = cudaGetLastError();
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to launch kernel (error code %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T>
__host__ void copyFromDeviceToHost(T *d_c, T *h_c, int M, int N) {
  size_t bytes = M * N * sizeof(T);
  cudaError_t err = cudaMemcpy(h_c, d_c, bytes, cudaMemcpyDeviceToHost);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy d_c to h_c (error code %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T>
__host__ void deallocateMemory(T *d_a, T *d_b, T *d_c) {
  cudaError_t err = cudaFree(d_a);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to free d_a (error code %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  cudaFree(d_b);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to free d_b (error code %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  cudaFree(d_c);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to free d_c (error code %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ void cleanUpDevice() {
  cudaError_t err = cudaDeviceReset();
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to clean up device (error code %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ std::tuple<int, int, int> parseCmdLineArgs(int argc, char *argv[]) {
  int M = 1024;
  int N = 1024;
  int K = 1024;

  for (int i = 1; i < argc; i++){
    std::string option(argv[i]);
    std::string value(argv[i+1]);
    i++;
    if (option.compare("-m") == 0) {
      M = std::stoi(value);
    }
    else if (option.compare("-n") == 0) {
      N = std::stoi(value);
    }
    else if (option.compare("-k") == 0) {
      K = std::stoi(value);
    }
  }
  return {M, N, K};
}

int main(int argc, char *argv[]) {
  std::tuple<int, int, int>parsedCmdLineArgsTuple = parseCmdLineArgs(argc, argv);
  int M = std::get<0>(parsedCmdLineArgsTuple);
  int N = std::get<1>(parsedCmdLineArgsTuple);
  int K = std::get<2>(parsedCmdLineArgsTuple);

  // allocate memory on host side
  int *h_a = (int *)malloc(M * K * sizeof(int));
  int *h_b = (int *)malloc(K * N * sizeof(int));
  int *h_c = (int *)malloc(M * N * sizeof(int));

  // initialize
  for (size_t i = 0; i < M; i++) {
    for (size_t j = 0; j < K; j++) {
      h_a[i * K + j] = rand() % 10;
    }
  }

  for (size_t i = 0; i < K; i++) {
    for (size_t j = 0; j < N; j++) {
      h_b[i * N + j] = rand() % 10;
    }
  }
  // allocate memory on device side
  int *d_a, *d_b, *d_c;
  cudaMalloc((int **)&d_a, M * K * sizeof(int));
  cudaMalloc((int **)&d_b, K * N * sizeof(int));
  cudaMalloc((int **)&d_c, M * N* sizeof(int));

  copyFromHostToDevice<int>(h_a, h_b, d_a, d_b, M, N, K);

  //cudaEvent_t start, stop;
  //float time;
  //cudaEventCreate(&start);
  //cudaEventCreate(&stop);
//
  //cudaEventRecord( start, 0 );

  executeKernel<int>(d_a, d_b, d_c, M, N, K);

  //cudaEventRecord( stop, 0 );
  //cudaEventSynchronize( stop );
//
  //cudaEventElapsedTime( &time, start, stop );
  //printf("Time taken for GEMM: %f ms\n", time);
  //cudaEventDestroy( start );
  //cudaEventDestroy( stop );

  //std::cout << "Performance: " << 2LL*N*N*N/(time * 1e-3 * 1e9) << " GFLOP/s\n";

  copyFromDeviceToHost<int>(d_c, h_c, M, N);
  //verifyResult<int>(h_a, h_b, h_c, M, N, K);
  deallocateMemory<int>(d_a, d_b, d_c);
  cleanUpDevice();
  return 0;
}

UsageError: unrecognized arguments: tính\New folder (2)


In [ ]:
%%bash
nvcc -o /content/build/naive_out /content/src/naive-main.cu -arch=sm_75 -Xptxas=-v
nvcc -ptx /content/src/naive-main.cu -arch=sm_75 -o /content/naive_out.ptx
nvcc -arch=sm_75 -cubin /content/src/naive-main.cu -o /content/naive.cubin
cuobjdump -sass naive.cubin
cd /content/build
./naive_out -m 1024 -n 1024 -k 1024 -p false



	code for sm_75
		Function : _Z25globaltimer_matmul_kernelIiEvPKT_S2_PS0_iibP23GlobalTimerThreadRecord
	.headerflags	@"EF_CUDA_TEXMODE_UNIFIED EF_CUDA_64BIT_ADDRESS EF_CUDA_SM75 EF_CUDA_VIRTUAL_SM(EF_CUDA_SM75)"
        /*0000*/                   MOV R1, c[0x0][0x28] ;                                /* 0x00000a0000017a02 */
                                                                                         /* 0x000fe40000000f00 */
        /*0010*/                   CS2R R2, SR_GLOBALTIMERLO ;                           /* 0x0000000000027805 */
                                                                                         /* 0x000fcc0000015200 */
        /*0020*/                   IABS R10, c[0x0][0x178] ;                             /* 0x00005e00000a7a13 */
                                                                                         /* 0x000fe20000000000 */
        /*0030*/                   S2R R5, SR_TID.X ;                                    /* 0x000000000

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z25globaltimer_matmul_kernelIiEvPKT_S2_PS0_iibP23GlobalTimerThreadRecord' for 'sm_75'
ptxas info    : Function properties for _Z25globaltimer_matmul_kernelIiEvPKT_S2_PS0_iibP23GlobalTimerThreadRecord
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 32 registers, 400 bytes cmem[0]
ptxas info    : Compiling entry function '_Z21clock64_matmul_kernelIiEvPKT_S2_PS0_iibP19Clock64ThreadRecord' for 'sm_75'
ptxas info    : Function properties for _Z21clock64_matmul_kernelIiEvPKT_S2_PS0_iibP19Clock64ThreadRecord
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 32 registers, 400 bytes cmem[0]


In [ ]:
%%bash
cd /content/build
ncu -f --set full --import-source on -o /content/naive naive_out -m 1024 -n 1024 -k 1024
ncu --print-details all --print-rule-details --set full ./naive_out -m 1024 -n 1024 -k 1024 -p false > /content/report/naive.report

==PROF== Connected to process 949 (/content/build/naive_out)
==PROF== Profiling "clock64_matmul_kernel" - 0: 0%....50%....100% - 30 passes
==PROF== Disconnected from process 949
==WARNING== No source files were imported. Check that the target application was compiled with -lineinfo.
==PROF== Report: /content/naive.ncu-rep


# Tiled (Block level) - 532.632 GFLOP/s

In [ ]:
%%writefile /content/src/tiled-main.cu

#include <algorithm>
#include <cassert>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <iostream>

template<typename T>
__host__ void verifyResult(T *h_a, T *h_b, T *h_c, int M, int N, int K) {
  for (int i = 0; i < M; i++) {
    for (int j = 0; j < N; j++) {
      T sum = 0;
      for (int k = 0; k < K; k++) {
        sum += h_a[i * K + k] * h_b[k * N + j];
      }
      assert(h_c[i * N + j] == sum);
    }
  }
  printf("Correct!");
}

// bM = 16, bN = 16, bK = 16 - non coalesced
// bM = 32, bN = 32, bK = 32 - coalesced
template<typename T, const size_t bM, const size_t bN, const size_t bK>
__global__ void gemm_kernel(T* d_a, T* d_b, T* d_c, size_t M, size_t N, size_t K) {
  assert(bM * bK == blockDim.x);
  assert(bK * bN == blockDim.x);

  // move blocktile to beginning of A's row and B's column
  const size_t cRow = blockIdx.y;
  const size_t cCol = blockIdx.x;

  d_a += cRow * bM * K;
  d_b += cCol * bN;
  d_c += cRow * bM * N + cCol * bN;

  // The total shared memory used is (bM * bK * 4 (bytes) + bK * bN * 4 (bytes))
  __shared__ T As[bM * bK];
  __shared__ T Bs[bK * bN];

  // At thread level
  const size_t threadCol = threadIdx.x % bN;
  const size_t threadRow = threadIdx.x / bN;

  T tmp = 0.0;
  for (int bkIdx = 0; bkIdx < K; bkIdx += bK) {

    As[threadRow * bK + threadCol] = d_a[threadRow * K + threadCol]; // is this coalesced? 32=yes, 16=no
    Bs[threadRow * bN + threadCol] = d_b[threadRow * N + threadCol];

    __syncthreads();

    d_a += bK;
    d_b += bK * N;

    for (size_t dotIdx = 0; dotIdx < bK; dotIdx++) {
      tmp += As[threadRow * bK + dotIdx] * Bs[dotIdx * bN + threadCol];
    }
    __syncthreads(); // barrier synchronization ~ synchronize threads within a block
    //printf("tmp: %f\n", tmp);
  }

  d_c[threadRow * N + threadCol] = tmp;
  //printf("d_c[%d * N + %d]: %f\n", threadRow, threadCol, d_c[threadRow * N + threadCol]);
}

template<typename T>
__host__ void copyFromHostToDevice(T* h_a, T* h_b, T* d_a, T* d_b, size_t M, size_t N , size_t K) {
  size_t a_bytes = sizeof(T) * M * K;
  size_t b_bytes = sizeof(T) * K * N;
  cudaError_t err = cudaMemcpy(d_a, h_a, a_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_a to d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaMemcpy(d_b, h_b, b_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_b to d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T, const size_t bM, const size_t bN, const size_t bK>
__host__ void executeKernel(T* d_a, T* d_b, T* d_c, size_t M, size_t N, size_t K) {
  dim3 block(bM * bK, 1, 1);
  dim3 grid((M + bM - 1) / bM, (N + bN - 1) / bN, 1);
  // Ways to affect occupancy
  // 1. Changing template parameter affects the shared memory size (bM, bN, bK)
  // 2. Changing block size
  gemm_kernel<T, bM, bN, bK><<<grid, block>>>(d_a, d_b, d_c, M, N, K);
  cudaDeviceSynchronize();
}

template<typename T>
__host__ void copyFromDeviceToHost(T* d_c, T* h_c, size_t M, size_t N) {
  size_t c_bytes = sizeof(T) * M * N;
  cudaError_t err = cudaMemcpy(h_c, d_c, c_bytes, cudaMemcpyDeviceToHost);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy from d_c to h_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T>
__host__ void deallocateMemory(T* d_a, T* d_b, T* d_c) {
  cudaError_t err = cudaFree(d_a);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_b);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_c);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ void cleanUpDevice() {
  cudaError_t err = cudaDeviceReset();
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to clean up device (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ std::tuple<int, int, int> parseCmdLineArgs(int argc, char *argv[]) {
  int M = 1024;
  int N = 1024;
  int K = 1024;

  for (int i = 1; i < argc; i++){
    std::string option(argv[i]);
    std::string value(argv[i+1]);
    i++;
    if (option.compare("-m") == 0) {
      M = std::stoi(value);
    }
    else if (option.compare("-n") == 0) {
      N = std::stoi(value);
    }
    else if (option.compare("-k") == 0) {
      K = std::stoi(value);
    }
  }
  return {M, N, K};
}

int main(int argc, char *argv[]) {
  std::tuple<int, int, int>parsedCmdLineArgsTuple = parseCmdLineArgs(argc, argv);
  int M = std::get<0>(parsedCmdLineArgsTuple);
  int N = std::get<1>(parsedCmdLineArgsTuple);
  int K = std::get<2>(parsedCmdLineArgsTuple);

  float* h_a = (float*)malloc(M * K * sizeof(float));
  float* h_b = (float*)malloc(K * N * sizeof(float));
  float* h_c = (float*)malloc(M * N * sizeof(float));

  // initialize
  for (size_t i = 0; i < M; i++) {
    for (size_t j = 0; j < K; j++) {
      h_a[i * K + j] = rand() % 10;
    }
  }

  for (size_t i = 0; i < K; i++) {
    for (size_t j = 0; j < N; j++) {
      h_b[i * N + j] = rand() % 10;
    }
  }

  // allocate memory on device side
  float *d_a, *d_b, *d_c;
  cudaMalloc((float **)&d_a, M * K * sizeof(float));
  cudaMalloc((float **)&d_b, K * N * sizeof(float));
  cudaMalloc((float **)&d_c, M * N * sizeof(float));

  copyFromHostToDevice<float>(h_a, h_b, d_a, d_b, M, N, K);

  //cudaEvent_t start, stop;
  //float time;
  //cudaEventCreate(&start);
  //cudaEventCreate(&stop);
//
  //cudaEventRecord( start, 0 );
  //executeKernel<float, 16, 16, 16>(d_a, d_b, d_c, M, N, K);
  executeKernel<float, 32, 32, 32>(d_a, d_b, d_c, M, N, K);

  //cudaEventRecord( stop, 0 );
  //cudaEventSynchronize( stop );
//
  //cudaEventElapsedTime( &time, start, stop );
  //printf("Time taken for GEMM: %f ms\n", time);
  //cudaEventDestroy( start );
  //cudaEventDestroy( stop );

  //std::cout << "Performance: " << 2LL*M*N*K/(time * 1e-3 * 1e9) << " GFLOP/s\n";

  copyFromDeviceToHost<float>(d_c, h_c, M, N);
  //verifyResult<float>(h_a, h_b, h_c, M, N, K);
  deallocateMemory<float>(d_a, d_b, d_c);
  cleanUpDevice();
  return 0;
}

Writing /content/src/tiled-main.cu


In [ ]:
%%bash
nvcc -o /content/build/tiled_out /content/src/tiled-main.cu -arch=sm_75 -Xptxas=-v
cd /content/build
./tiled_out -m 1024 -n 1024 -k 1024

ptxas info    : 240 bytes gmem
ptxas info    : Compiling entry function '_Z11gemm_kernelIfLm32ELm32ELm32EEvPT_S1_S1_mmm' for 'sm_75'
ptxas info    : Function properties for _Z11gemm_kernelIfLm32ELm32ELm32EEvPT_S1_S1_mmm
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 49 registers, 8192 bytes smem, 400 bytes cmem[0]


In [ ]:
%%bash
cd /content/build
ncu -f --set full --import-source on -o  /content/tiled_out tiled_out -m 1024 -n 1024 -k 1024
ncu --print-details all --print-rule-details --set full ./tiled_out -m 1024 -n 1024 -k 1024 > /content/report/tiled.report

==PROF== Connected to process 8017 (/content/build/tiled_out)
==PROF== Profiling "gemm_kernel" - 0: 0%....50%....100% - 31 passes
==PROF== Disconnected from process 8017
==WARNING== No source files were imported. Check that the target application was compiled with -lineinfo.
==PROF== Report: /content/tiled_out.ncu-rep


# Tiled (1D - ILP) - 971.392 GFLOP/s

In [ ]:
%%writefile /content/src/tiled-ilp-1D-main.cu

#include <algorithm>
#include <cassert>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <iostream>

template<typename T>
__host__ void verifyResult(T *h_a, T *h_b, T *h_c, int M, int N, int K) {
  for (int i = 0; i < M; i++) {
    for (int j = 0; j < N; j++) {
      T sum = 0;
      for (int k = 0; k < K; k++) {
        sum += h_a[i * K + k] * h_b[k * N + j];
      }
      //if (i == 0 && j == 0) {
      //  printf("sum: %f, h_c[%d * K + %d]: %f\n", sum, i, j, h_c[i * K + j]);
      //}
      assert(h_c[i * N + j] == sum);
    }
  }
  printf("Correct!\n");
}

template<typename T, size_t BM, size_t BN, size_t BK, size_t TM>
__global__ void gemm_kernel(T* A, T* B, T* C, size_t M, size_t N, size_t K) {
  const uint cRow = blockIdx.y;
  const uint cCol = blockIdx.x;

  // each warp will calculate 32*TM elements, with 32 being the columnar dim.
  const int threadCol = threadIdx.x % BN;
  const int threadRow = threadIdx.x / BN;

  // allocate space for the current blocktile in SMEM
  __shared__ float As[BM * BK];
  __shared__ float Bs[BK * BN];

  // Move blocktile to beginning of A's row and B's column
  A += cRow * BM * K;
  B += cCol * BN;
  C += cRow * BM * N + cCol * BN;

  // todo: adjust this to each thread to load multiple entries and
  // better exploit the cache sizes
  assert(BM * BK == blockDim.x);
  assert(BN * BK == blockDim.x);
  const uint innerColA = threadIdx.x % BK; // warp-level GMEM coalescing
  const uint innerRowA = threadIdx.x / BK;
  const uint innerColB = threadIdx.x % BN; // warp-level GMEM coalescing
  const uint innerRowB = threadIdx.x / BN;

  // allocate thread-local cache for results in registerfile
  float threadResults[TM] = {0.0};

  // outer loop over block tiles
  for (uint bkIdx = 0; bkIdx < K; bkIdx += BK) {
    // populate the SMEM caches
    As[innerRowA * BK + innerColA] = A[innerRowA * K + innerColA];
    Bs[innerRowB * BN + innerColB] = B[innerRowB * N + innerColB];
    __syncthreads();

    // advance blocktile
    A += BK;
    B += BK * N;

    // calculate per-thread results
    for (uint dotIdx = 0; dotIdx < BK; ++dotIdx) {
      // we make the dotproduct loop the outside loop, which facilitates
      // reuse of the Bs entry, which we can cache in a tmp var.
      float tmpB = Bs[dotIdx * BN + threadCol];
      for (uint resIdx = 0; resIdx < TM; ++resIdx) {
        threadResults[resIdx] +=
            As[(threadRow * TM + resIdx) * BK + dotIdx] * tmpB;
      }
    }
    __syncthreads();
  }

  // write out the results
  for (uint resIdx = 0; resIdx < TM; ++resIdx) {
    C[(threadRow * TM + resIdx) * N + threadCol] = threadResults[resIdx];
  }
}

template<typename T>
__host__ void copyFromHostToDevice(T* h_a, T* h_b, T* d_a, T* d_b, size_t M, size_t N , size_t K) {
  size_t a_bytes = sizeof(T) * M * K;
  size_t b_bytes = sizeof(T) * K * N;
  cudaError_t err = cudaMemcpy(d_a, h_a, a_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_a to d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaMemcpy(d_b, h_b, b_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_b to d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T, const uint BM, const uint BN, const uint BK, const uint TM>
__host__ void executeKernel(T* d_a, T* d_b, T* d_c, size_t M, size_t N, size_t K) {
  dim3 block((BM * BN) / TM);
  dim3 grid((M + BM - 1) / BM, (N + BN - 1) / BN);
  gemm_kernel<T, BM, BN, BK, TM><<<grid, block>>>(d_a, d_b, d_c, M, N, K);
  cudaDeviceSynchronize();
}

template<typename T>
__host__ void copyFromDeviceToHost(T* d_c, T* h_c, size_t M, size_t N) {
  size_t c_bytes = sizeof(T) * M * N;
  cudaError_t err = cudaMemcpy(h_c, d_c, c_bytes, cudaMemcpyDeviceToHost);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy from d_c to h_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T>
__host__ void deallocateMemory(T* d_a, T* d_b, T* d_c) {
  cudaError_t err = cudaFree(d_a);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_b);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_c);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ void cleanUpDevice() {
  cudaError_t err = cudaDeviceReset();
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to clean up device (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ std::tuple<int, int, int> parseCmdLineArgs(int argc, char *argv[]) {
  int M = 1024;
  int N = 1024;
  int K = 1024;

  for (int i = 1; i < argc; i++){
    std::string option(argv[i]);
    std::string value(argv[i+1]);
    i++;
    if (option.compare("-m") == 0) {
      M = std::stoi(value);
    }
    else if (option.compare("-n") == 0) {
      N = std::stoi(value);
    }
    else if (option.compare("-k") == 0) {
      K = std::stoi(value);
    }
  }
  return {M, N, K};
}

int main(int argc, char *argv[]) {
std::tuple<int, int, int>parsedCmdLineArgsTuple = parseCmdLineArgs(argc, argv);
  int M = std::get<0>(parsedCmdLineArgsTuple);
  int N = std::get<1>(parsedCmdLineArgsTuple);
  int K = std::get<2>(parsedCmdLineArgsTuple);
  float* h_a = (float*)malloc(M * K * sizeof(float));
  float* h_b = (float*)malloc(K * N * sizeof(float));
  float* h_c = (float*)malloc(M * N * sizeof(float));

  // initialize
  for (size_t i = 0; i < M; i++) {
    for (size_t j = 0; j < K; j++) {
      h_a[i * K + j] = rand() % 10;
    }
  }

  for (size_t i = 0; i < K; i++) {
    for (size_t j = 0; j < N; j++) {
      h_b[i * N + j] = rand() % 10;
    }
  }

  // allocate memory on device side
  float *d_a, *d_b, *d_c;
  cudaMalloc((float **)&d_a, M * K * sizeof(float));
  cudaMalloc((float **)&d_b, K * N * sizeof(float));
  cudaMalloc((float **)&d_c, M * N * sizeof(float));

  copyFromHostToDevice<float>(h_a, h_b, d_a, d_b, M, N, K);

  //cudaEvent_t start, stop;
  //float time;
  //cudaEventCreate(&start);
  //cudaEventCreate(&stop);
//
  //cudaEventRecord( start, 0 );

  executeKernel<float, 64, 64, 8, 8>(d_a, d_b, d_c, M, N, K);

  //cudaEventRecord( stop, 0 );
  //cudaEventSynchronize( stop );
//
  //cudaEventElapsedTime( &time, start, stop );
  //printf("Time taken for GEMM: %f ms\n", time);
  //cudaEventDestroy( start );
  //cudaEventDestroy( stop );
//
  //std::cout << "Performance: " << 2LL*M*N*K/(time * 1e-3 * 1e9) << " GFLOP/s\n";

  copyFromDeviceToHost<float>(d_c, h_c, M, N);
  //verifyResult<float>(h_a, h_b, h_c, M, N, K);
  deallocateMemory<float>(d_a, d_b, d_c);
  cleanUpDevice();
  return 0;
}

Writing /content/src/tiled-ilp-1D-main.cu


In [ ]:
%%bash
nvcc -o /content/build/tiled_ilp_1D_out /content/src/tiled-ilp-1D-main.cu -arch=sm_75 -Xptxas=-v
cd /content/build
./tiled_ilp_1D_out -m 1024 -n 1024 -k 1024

ptxas info    : 270 bytes gmem
ptxas info    : Compiling entry function '_Z11gemm_kernelIfLm64ELm64ELm8ELm8EEvPT_S1_S1_mmm' for 'sm_75'
ptxas info    : Function properties for _Z11gemm_kernelIfLm64ELm64ELm8ELm8EEvPT_S1_S1_mmm
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 70 registers, 4096 bytes smem, 400 bytes cmem[0]


In [ ]:
%%bash
cd /content/build
ncu -f --set full --import-source on -o  /content/tiled_ilp_1D_out tiled_ilp_1D_out -m 1024 -n 1024 -k 1024
ncu --print-details all --print-rule-details --set full ./tiled_ilp_1D_out -m 1024 -n 1024 -k 1024 > /content/report/tiled-ilp.report

==PROF== Connected to process 8740 (/content/build/tiled_ilp_1D_out)
==PROF== Profiling "gemm_kernel" - 0: 0%....50%....100% - 31 passes
==PROF== Disconnected from process 8740
==WARNING== No source files were imported. Check that the target application was compiled with -lineinfo.
==PROF== Report: /content/tiled_ilp_1D_out.ncu-rep


# Tiled (2D-ILP) - 1867.65 GFLOP/s

In [ ]:
%%writefile /content/src/tiled-ilp-2D-main.cu

#include <algorithm>
#include <cassert>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <iostream>

template<typename T>
__host__ void verifyResult(T *h_a, T *h_b, T *h_c, int M, int N, int K) {
  for (int i = 0; i < M; i++) {
    for (int j = 0; j < N; j++) {
      T sum = 0;
      for (int k = 0; k < K; k++) {
        sum += h_a[i * K + k] * h_b[k * N + j];
      }
      //printf("sum: %f, h_c[%d * K + %d]: %f\n", sum, i, j, h_c[i * K + j]);
      assert(h_c[i * N + j] == sum);
    }
  }
  printf("Correct!\n");
}

// BM = 128, BN = 128, BK = 16, TM = 8, TN = 8
template<typename T, size_t BM, size_t BN, size_t BK, size_t TM, size_t TN>
__global__ void gemm_kernel(T* A, T* B, T* C, size_t M, size_t N, size_t K) {
  const uint totalResultsBlocktile = BM * BN; // 128 * 128
  // A thread is responsible for calculating TM*TN elements in the blocktile
  const uint numThreadsBlocktile = totalResultsBlocktile / (TM * TN); // 128 * 128 / (8 * 8) = 256

  // ResultsPerBlock / ResultsPerThread == ThreadsPerBlock
  assert(numThreadsBlocktile == blockDim.x);

  // Calculate thread local index within a block with respect to matrix C
  const int threadCol = threadIdx.x % (BN / TN);
  const int threadRow = threadIdx.x / (BN / TN);

  // allocate space for the current blocktile in smem
  __shared__ float As[BM * BK]; // 128 * 16
  __shared__ float Bs[BK * BN]; // 16 * 128

  // Move blocktile to beginning of A's row and B's column
  A += blockIdx.y * BM * K;
  B += blockIdx.x * BN;
  C += blockIdx.y * BM * N + blockIdx.x * BN;

  // calculating the local indices w.r.t matrix A that this thread will load into SMEM
  const uint innerRowA = threadIdx.x / BK;
  const uint innerColA = threadIdx.x % BK;
  // calculates the number of rows of As that are being loaded in a single step
  // by a single block
  const uint strideA = numThreadsBlocktile / BK;    // 256/16 = 16
  // calculating the local indices w.r.t matrix B that this thread will load into SMEM
  const uint innerRowB = threadIdx.x / BN;
  const uint innerColB = threadIdx.x % BN;
  const uint strideB = numThreadsBlocktile / BN;    // 256/128 = 2

  // allocate thread-local cache for results in registerfile
  float threadResults[TM * TN] = {0.0};
  // register caches for As and Bs
  float regM[TM] = {0.0};
  float regN[TN] = {0.0};

  // outer-most loop over block tiles
  for (uint bkIdx = 0; bkIdx < K; bkIdx += BK) {
    // populate the SMEM caches
    for (uint loadOffset = 0; loadOffset < BM; loadOffset += strideA) {
      As[(innerRowA + loadOffset) * BK + innerColA] =
          A[(innerRowA + loadOffset) * K + innerColA];
    }
    for (uint loadOffset = 0; loadOffset < BK; loadOffset += strideB) {
      Bs[(innerRowB + loadOffset) * BN + innerColB] =
          B[(innerRowB + loadOffset) * N + innerColB];
    }
    __syncthreads();

    // advance blocktile
    A += BK;     // move BK columns to right
    B += BK * N; // move BK rows down

    // calculate per-thread results
    // sequential reduction
    for (uint dotIdx = 0; dotIdx < BK; ++dotIdx) {
      // block into registers
      for (uint i = 0; i < TM; ++i) {
        regM[i] = As[(threadRow * TM + i) * BK + dotIdx];
      }
      for (uint i = 0; i < TN; ++i) {
        regN[i] = Bs[dotIdx * BN + threadCol * TN + i];
      }
      for (uint resIdxM = 0; resIdxM < TM; ++resIdxM) {
        for (uint resIdxN = 0; resIdxN < TN; ++resIdxN) {
          threadResults[resIdxM * TN + resIdxN] +=
              regM[resIdxM] * regN[resIdxN];
        }
      }
    }
    __syncthreads();
  }

  // write out the results
  for (uint resIdxM = 0; resIdxM < TM; ++resIdxM) {
    for (uint resIdxN = 0; resIdxN < TN; ++resIdxN) {
      C[(threadRow * TM + resIdxM) * N + threadCol * TN + resIdxN] = threadResults[resIdxM * TN + resIdxN];
    }
  }
}

template<typename T>
__host__ void copyFromHostToDevice(T* h_a, T* h_b, T* d_a, T* d_b, size_t M, size_t N , size_t K) {
  size_t a_bytes = sizeof(T) * M * K;
  size_t b_bytes = sizeof(T) * K * N;
  cudaError_t err = cudaMemcpy(d_a, h_a, a_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_a to d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaMemcpy(d_b, h_b, b_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_b to d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T, const uint BM, const uint BN, const uint BK, const uint TM, const uint TN>
__host__ void executeKernel(T* d_a, T* d_b, T* d_c, size_t M, size_t N, size_t K) {
  dim3 block((BM * BN) / (TM * TN), 1, 1);
  dim3 grid((M + BM - 1) / BM, (N + BN - 1) / BN, 1);
  gemm_kernel<T, BM, BN, BK, TM, TN><<<grid, block>>>(d_a, d_b, d_c, M, N, K);
  cudaDeviceSynchronize();
}

template<typename T>
__host__ void copyFromDeviceToHost(T* d_c, T* h_c, size_t M, size_t N) {
  size_t c_bytes = sizeof(T) * M * N;
  cudaError_t err = cudaMemcpy(h_c, d_c, c_bytes, cudaMemcpyDeviceToHost);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy from d_c to h_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T>
__host__ void deallocateMemory(T* d_a, T* d_b, T* d_c) {
  cudaError_t err = cudaFree(d_a);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_b);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_c);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ void cleanUpDevice() {
  cudaError_t err = cudaDeviceReset();
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to clean up device (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ std::tuple<int, int, int> parseCmdLineArgs(int argc, char *argv[]) {
  int M = 1024;
  int N = 1024;
  int K = 1024;

  for (int i = 1; i < argc; i++){
    std::string option(argv[i]);
    std::string value(argv[i+1]);
    i++;
    if (option.compare("-m") == 0) {
      M = std::stoi(value);
    }
    else if (option.compare("-n") == 0) {
      N = std::stoi(value);
    }
    else if (option.compare("-k") == 0) {
      K = std::stoi(value);
    }
  }
  return {M, N, K};
}

int main(int argc, char *argv[]) {
std::tuple<int, int, int>parsedCmdLineArgsTuple = parseCmdLineArgs(argc, argv);
  int M = std::get<0>(parsedCmdLineArgsTuple);
  int N = std::get<1>(parsedCmdLineArgsTuple);
  int K = std::get<2>(parsedCmdLineArgsTuple);

  float* h_a = (float*)malloc(M * K * sizeof(float));
  float* h_b = (float*)malloc(K * N * sizeof(float));
  float* h_c = (float*)malloc(M * N * sizeof(float));

  // initialize
  for (size_t i = 0; i < M; i++) {
    for (size_t j = 0; j < K; j++) {
      h_a[i * K + j] = rand() % 10;
    }
  }

  for (size_t i = 0; i < K; i++) {
    for (size_t j = 0; j < N; j++) {
      h_b[i * N + j] = rand() % 10;
    }
  }

  // allocate memory on device side
  float *d_a, *d_b, *d_c;
  cudaMalloc((float **)&d_a, M * K * sizeof(float));
  cudaMalloc((float **)&d_b, K * N * sizeof(float));
  cudaMalloc((float **)&d_c, M * N * sizeof(float));

  copyFromHostToDevice<float>(h_a, h_b, d_a, d_b, M, N, K);

  //cudaEvent_t start, stop;
  //float time;
  //cudaEventCreate(&start);
  //cudaEventCreate(&stop);
//
  //cudaEventRecord( start, 0 );

  executeKernel<float, 128, 128, 16, 8, 8>(d_a, d_b, d_c, M, N, K);

  //cudaEventRecord( stop, 0 );
  //cudaEventSynchronize( stop );
//
  //cudaEventElapsedTime( &time, start, stop );
  //printf("Time taken for GEMM: %f ms\n", time);
  //cudaEventDestroy( start );
  //cudaEventDestroy( stop );
//
  //std::cout << "Performance: " << 2LL*M*N*K/(time * 1e-3 * 1e9) << " GFLOP/s\n";

  copyFromDeviceToHost<float>(d_c, h_c, M, N);
  //verifyResult<float>(h_a, h_b, h_c, M, N, K);
  deallocateMemory<float>(d_a, d_b, d_c);
  cleanUpDevice();
  return 0;
}

Writing /content/src/tiled-ilp-2D-main.cu


In [ ]:
%%bash
nvcc -o /content/build/tiled_ilp_2D_out /content/src/tiled-ilp-2D-main.cu -arch=sm_75 -Xptxas=-v
cd /content/build
./tiled_ilp_2D_out -m 1024 -n 1024 -k 1024

ptxas info    : 287 bytes gmem
ptxas info    : Compiling entry function '_Z11gemm_kernelIfLm128ELm128ELm16ELm8ELm8EEvPT_S1_S1_mmm' for 'sm_75'
ptxas info    : Function properties for _Z11gemm_kernelIfLm128ELm128ELm16ELm8ELm8EEvPT_S1_S1_mmm
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 126 registers, 16384 bytes smem, 400 bytes cmem[0]


In [ ]:
%%bash
cd /content/build
ncu -f --set full --import-source on -o  /content/tiled_ilp_2D tiled_ilp_2D_out -m 1024 -n 1024 -k 1024
ncu --print-details all --print-rule-details --set full ./tiled_ilp_2D_out -m 1024 -n 1024 -k 1024 > /content/report/tiled-ilp-2D.report

==PROF== Connected to process 9034 (/content/build/tiled_ilp_2D_out)
==PROF== Profiling "gemm_kernel" - 0: 0%....50%....100% - 31 passes
==PROF== Disconnected from process 9034
==WARNING== No source files were imported. Check that the target application was compiled with -lineinfo.
==PROF== Report: /content/tiled_ilp_2D.ncu-rep


# Vectorized - 1745.28 GFLOP/s

In [ ]:
%%writefile /content/src/vectorized-main.cu

#include <algorithm>
#include <cassert>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <iostream>

template<typename T>
__host__ void verifyResult(T *h_a, T *h_b, T *h_c, int M, int N, int K) {
  for (int i = 0; i < M; i++) {
    for (int j = 0; j < N; j++) {
      T sum = 0;
      for (int k = 0; k < K; k++) {
        sum += h_a[i * K + k] * h_b[k * N + j];
      }
      //printf("sum: %f, h_c[%d * K + %d]: %f\n", sum, i, j, h_c[i * K + j]);
      assert(h_c[i * N + j] == sum);
    }
  }
  printf("Correct!\n");
}

// BM = 128, BN = 128, BK = 8, TM = 8, TN = 8
template<typename T, size_t BM, size_t BN, size_t BK, size_t TM, size_t TN>
__global__ void gemm_kernel(T* A, T* B, T* C, size_t M, size_t N, size_t K) {
  const uint totalResultsBlocktile = BM * BN;
  // A thread is responsible for calculating TM*TN elements in the blocktile
  const uint numThreadsBlocktile = totalResultsBlocktile / (TM * TN);

  // ResultsPerBlock / ResultsPerThread == ThreadsPerBlock
  assert(numThreadsBlocktile == blockDim.x);

  // Calculate thread local index within a block with respect to matrix C
  const int threadCol = threadIdx.x % (BN / TN);
  const int threadRow = threadIdx.x / (BN / TN);

  // allocate space for the current blocktile in smem
  __shared__ float As[BM * BK];
  __shared__ float Bs[BK * BN];

  // Move blocktile to beginning of A's row and B's column
  A += blockIdx.y * BM * K;
  B += blockIdx.x * BN;
  C += blockIdx.y * BM * N + blockIdx.x * BN;

  // calculating the local indices w.r.t matrix A that this thread will load into SMEM
  const uint innerRowA = threadIdx.x / (BK / 4);
  const uint innerColA = threadIdx.x % (BK / 4);
  // calculates the number of rows of As that are being loaded in a single step
  // by a single block
  const uint innerRowB = threadIdx.x / (BN / 4);
  const uint innerColB = threadIdx.x % (BN / 4);

  // allocate thread-local cache for results in registerfile
  float threadResults[TM * TN] = {0.0};
  // register caches for As and Bs
  float regM[TM] = {0.0};
  float regN[TN] = {0.0};

  // outer-most loop over block tiles
  for (uint bkIdx = 0; bkIdx < K; bkIdx += BK) {

    // populate the SMEM caches
    reinterpret_cast<float4 *>(&As[innerRowA * BK + innerColA * 4])[0] =
        reinterpret_cast<float4 *>(&A[innerRowA * K + innerColA * 4])[0];

    reinterpret_cast<float4 *>(&Bs[innerRowB * BN + innerColB * 4])[0] =
        reinterpret_cast<float4 *>(&B[innerRowB * N + innerColB * 4])[0];

    __syncthreads();

    // advance blocktile
    A += BK;     // move BK columns to right
    B += BK * N; // move BK rows down

    // calculate per-thread results
    // sequential reduction
    for (uint dotIdx = 0; dotIdx < BK; ++dotIdx) {
      // block into registers
      for (uint i = 0; i < TM; ++i) {
        regM[i] = As[(threadRow * TM + i) * BK + dotIdx];
      }
      for (uint i = 0; i < TN; ++i) {
        regN[i] = Bs[dotIdx * BN + threadCol * TN + i];
      }
      for (uint resIdxM = 0; resIdxM < TM; ++resIdxM) {
        for (uint resIdxN = 0; resIdxN < TN; ++resIdxN) {
          threadResults[resIdxM * TN + resIdxN] +=
              regM[resIdxM] * regN[resIdxN];
        }
      }
    }
    __syncthreads();
  }

  // write out the results
  for (uint resIdxM = 0; resIdxM < TM; resIdxM+=1) {
    for (uint resIdxN = 0; resIdxN < TN; resIdxN+=4) {
      // load C vector into registers
      float4 tmp = reinterpret_cast<float4 *>(
          &C[(threadRow * TM + resIdxM) * N + threadCol * TN + resIdxN])[0];
      // perform GEMM update in reg
      tmp.x = threadResults[resIdxM * TN + resIdxN];
      tmp.y = threadResults[resIdxM * TN + resIdxN + 1];
      tmp.z = threadResults[resIdxM * TN + resIdxN + 2];
      tmp.w = threadResults[resIdxM * TN + resIdxN + 3];
      // write back
      reinterpret_cast<float4 *>(
          &C[(threadRow * TM + resIdxM) * N + threadCol * TN + resIdxN])[0] =
          tmp;
    }
  }
}

template<typename T>
__host__ void copyFromHostToDevice(T* h_a, T* h_b, T* d_a, T* d_b, size_t M, size_t N , size_t K) {
  size_t a_bytes = sizeof(T) * M * K;
  size_t b_bytes = sizeof(T) * K * N;
  cudaError_t err = cudaMemcpy(d_a, h_a, a_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_a to d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaMemcpy(d_b, h_b, b_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_b to d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T, const uint BM, const uint BN, const uint BK, const uint TM, const uint TN>
__host__ void executeKernel(T* d_a, T* d_b, T* d_c, size_t M, size_t N, size_t K) {
  dim3 block((BM * BN) / (TM * TN), 1, 1);
  dim3 grid((M + BM - 1) / BM, (N + BN - 1) / BN, 1);
  gemm_kernel<T, BM, BN, BK, TM, TN><<<grid, block>>>(d_a, d_b, d_c, M, N, K);
  cudaDeviceSynchronize();
}

template<typename T>
__host__ void copyFromDeviceToHost(T* d_c, T* h_c, size_t M, size_t N) {
  size_t c_bytes = sizeof(T) * M * N;
  cudaError_t err = cudaMemcpy(h_c, d_c, c_bytes, cudaMemcpyDeviceToHost);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy from d_c to h_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T>
__host__ void deallocateMemory(T* d_a, T* d_b, T* d_c) {
  cudaError_t err = cudaFree(d_a);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_b);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_c);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ void cleanUpDevice() {
  cudaError_t err = cudaDeviceReset();
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to clean up device (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ std::tuple<int, int, int> parseCmdLineArgs(int argc, char *argv[]) {
  int M = 1024;
  int N = 1024;
  int K = 1024;

  for (int i = 1; i < argc; i++){
    std::string option(argv[i]);
    std::string value(argv[i+1]);
    i++;
    if (option.compare("-m") == 0) {
      M = std::stoi(value);
    }
    else if (option.compare("-n") == 0) {
      N = std::stoi(value);
    }
    else if (option.compare("-k") == 0) {
      K = std::stoi(value);
    }
  }
  return {M, N, K};
}

int main(int argc, char *argv[]) {
std::tuple<int, int, int>parsedCmdLineArgsTuple = parseCmdLineArgs(argc, argv);
  int M = std::get<0>(parsedCmdLineArgsTuple);
  int N = std::get<1>(parsedCmdLineArgsTuple);
  int K = std::get<2>(parsedCmdLineArgsTuple);

  float* h_a = (float*)malloc(M * K * sizeof(float));
  float* h_b = (float*)malloc(K * N * sizeof(float));
  float* h_c = (float*)malloc(M * N * sizeof(float));

  // initialize
  for (size_t i = 0; i < M; i++) {
    for (size_t j = 0; j < K; j++) {
      h_a[i * K + j] = rand() % 10;
    }
  }

  for (size_t i = 0; i < K; i++) {
    for (size_t j = 0; j < N; j++) {
      h_b[i * N + j] = rand() % 10;
    }
  }

  // allocate memory on device side
  float *d_a, *d_b, *d_c;
  cudaMalloc((float **)&d_a, M * K * sizeof(float));
  cudaMalloc((float **)&d_b, K * N * sizeof(float));
  cudaMalloc((float **)&d_c, M * N * sizeof(float));

  copyFromHostToDevice<float>(h_a, h_b, d_a, d_b, M, N, K);

  //cudaEvent_t start, stop;
  //float time;
  //cudaEventCreate(&start);
  //cudaEventCreate(&stop);
//
  //cudaEventRecord( start, 0 );

  executeKernel<float, 128, 128, 8, 8, 8>(d_a, d_b, d_c, M, N, K);

  //cudaEventRecord( stop, 0 );
  //cudaEventSynchronize( stop );
//
  //cudaEventElapsedTime( &time, start, stop );
  //printf("Time taken for GEMM: %f ms\n", time);
  //cudaEventDestroy( start );
  //cudaEventDestroy( stop );
//
  //std::cout << "Performance: " << 2LL*M*N*K/(time * 1e-3 * 1e9) << " GFLOP/s\n";

  copyFromDeviceToHost<float>(d_c, h_c, M, N);
  //verifyResult<float>(h_a, h_b, h_c, M, N, K);
  deallocateMemory<float>(d_a, d_b, d_c);
  cleanUpDevice();
  return 0;
}

Writing /content/src/vectorized-main.cu


In [ ]:
%%bash
nvcc -o /content/build/vectorized_out /content/src/vectorized-main.cu -arch=sm_75 -Xptxas=-v
cd /content/build
./vectorized_out -m 1024 -n 1024 -k 1024

ptxas info    : 284 bytes gmem
ptxas info    : Compiling entry function '_Z11gemm_kernelIfLm128ELm128ELm8ELm8ELm8EEvPT_S1_S1_mmm' for 'sm_75'
ptxas info    : Function properties for _Z11gemm_kernelIfLm128ELm128ELm8ELm8ELm8EEvPT_S1_S1_mmm
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 128 registers, 8192 bytes smem, 400 bytes cmem[0]


In [ ]:
%%bash
cd /content/build
ncu -f --set full --import-source on -o  /content/vectorized vectorized_out -m 1024 -n 1024 -k 1024
ncu --print-details all --print-rule-details --set full ./vectorized_out -m 1024 -n 1024 -k 1024 > /content/report/vectorized.report

==PROF== Connected to process 9324 (/content/build/vectorized_out)
==PROF== Profiling "gemm_kernel" - 0: 0%....50%....100% - 31 passes
==PROF== Disconnected from process 9324
==WARNING== No source files were imported. Check that the target application was compiled with -lineinfo.
==PROF== Report: /content/vectorized.ncu-rep


# Warp Tiled + CUDA Core - 2492.09 GFLOP/s

In [ ]:
%%writefile /content/src/warp-tiled-cc-main.cu

#include <algorithm>
#include <cassert>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <iostream>
const int WARPSIZE = 32;

template<typename T>
__host__ void verifyResult(T *h_a, T *h_b, T *h_c, int M, int N, int K) {
  for (int i = 0; i < M; i++) {
    for (int j = 0; j < N; j++) {
      T sum = 0;
      for (int k = 0; k < K; k++) {
        sum += h_a[i * K + k] * h_b[k * N + j];
      }
      //if (i==0 && j == 0){
      //  printf("sum: %f, h_c[%d * K + %d]: %f\n", sum, i, j, h_c[i * K + j]);
      //}
      assert(h_c[i * N + j] == sum);
    }
  }
  printf("Correct!\n");
}

template <const int BM, const int BN, const int BK, const int rowStrideA,
          const int rowStrideB>
__device__ void loadFromGmem(int N, int K, const float *A, const float *B,
                             float *As, float *Bs, int innerRowA, int innerColA,
                             int innerRowB, int innerColB) {
  for (uint offset = 0; offset + rowStrideA <= BM; offset += rowStrideA) {
    //reinterpret_cast<float4 *>(&As[(innerRowA + offset) * BK + innerColA * 4])[0] =
    //    reinterpret_cast<const float4 *>(&A[(innerRowA + offset) * K + innerColA * 4])[0];
    const float4 tmp = reinterpret_cast<const float4 *>(
        &A[(innerRowA + offset) * K + innerColA * 4])[0];
    As[(innerColA * 4 + 0) * BM + innerRowA + offset] = tmp.x;
    As[(innerColA * 4 + 1) * BM + innerRowA + offset] = tmp.y;
    As[(innerColA * 4 + 2) * BM + innerRowA + offset] = tmp.z;
    As[(innerColA * 4 + 3) * BM + innerRowA + offset] = tmp.w;
  }

  for (uint offset = 0; offset + rowStrideB <= BK; offset += rowStrideB) {
    reinterpret_cast<float4 *>(&Bs[(innerRowB + offset) * BN + innerColB * 4])[0] =
        reinterpret_cast<const float4 *>(&B[(innerRowB + offset) * N + innerColB * 4])[0];
  }
}

template <const int BM, const int BN, const int BK, const int WM, const int WN,
          const int WMITER, const int WNITER, const int WSUBM, const int WSUBN,
          const int TM, const int TN>
__device__ void
processFromSmem(float *regM, float *regN, float *threadResults, const float *As,
                const float *Bs, const uint warpRow, const uint warpCol,
                const uint threadRowInWarp, const uint threadColInWarp) {
  for (uint dotIdx = 0; dotIdx < BK; ++dotIdx) {
    // populate registers for whole warptile
    for (uint wSubRowIdx = 0; wSubRowIdx < WMITER; ++wSubRowIdx) {
      for (uint i = 0; i < TM; ++i) {
        //regM[wSubRowIdx * TM + i] =
        //    As[(warpRow * WM + wSubRowIdx * WSUBM +
        //       threadRowInWarp * TM + i) * BK + dotIdx];
        regM[wSubRowIdx * TM + i] =
            As[(warpRow * WM + wSubRowIdx * WSUBM +
               threadRowInWarp * TM + i) + dotIdx * BM];
      }
    }
    for (uint wSubColIdx = 0; wSubColIdx < WNITER; ++wSubColIdx) {
      for (uint i = 0; i < TN; ++i) {
        regN[wSubColIdx * TN + i] =
            Bs[(dotIdx * BN) + warpCol * WN + wSubColIdx * WSUBN +
               threadColInWarp * TN + i];
      }
    }

    for (uint wSubRowIdx = 0; wSubRowIdx < WMITER; ++wSubRowIdx) {
      for (uint wSubColIdx = 0; wSubColIdx < WNITER; ++wSubColIdx) {
        // calculate per-thread results
        for (uint resIdxM = 0; resIdxM < TM; ++resIdxM) {
          for (uint resIdxN = 0; resIdxN < TN; ++resIdxN) {
            threadResults[(wSubRowIdx * TM + resIdxM) * (WNITER * TN) +
                          (wSubColIdx * TN) + resIdxN] +=
                regM[wSubRowIdx * TM + resIdxM] *
                regN[wSubColIdx * TN + resIdxN];
          }
        }
      }
    }
  }
}

// BM = 128, BN = 128, BK = 16, TM = 8, TN = 4, WM = 64, WN = 64, WNITER = 4
template<typename T, size_t BM, size_t BN, size_t BK, size_t TM, size_t TN, size_t WM, size_t WN, size_t WNITER>
__global__ void gemm_kernel(T* A, T* B, T* C, size_t M, size_t N, size_t K) {
  const uint NUM_THREADS = 128;
  const uint cRow = blockIdx.y;
  const uint cCol = blockIdx.x;

  // Placement of the warp in the threadblock tile
  const uint warpIdx = threadIdx.x / WARPSIZE; // the warp this thread is in
  const uint warpCol = warpIdx % (BN / WN);
  const uint warpRow = warpIdx / (BN / WN);

  // size of warp subtile
  constexpr uint WMITER = (WM * WN) / (WARPSIZE * TM * TN * WNITER);
  constexpr uint WSUBM = WM / WMITER; // 64/4=16
  constexpr uint WSUBN = WN / WNITER; // 64/1=64

  // Placement of the thread in the warp subtile
  const uint threadIdxInWarp = threadIdx.x % WARPSIZE;
  const uint threadColInWarp = threadIdxInWarp % (WSUBN / TN);
  const uint threadRowInWarp = threadIdxInWarp / (WSUBN / TN);

  // allocate space for the current blocktile in SMEM
  __shared__ float As[BM * BK];
  __shared__ float Bs[BK * BN];

  // Move blocktile to beginning of A's row and B's column
  A += cRow * BM * K;
  B += cCol * BN;
  // Move C_ptr to warp's output tile
  C += (cRow * BM + warpRow * WM) * N + cCol * BN + warpCol * WN;

  // calculating the indices that this thread will load into SMEM
  // we'll load 128bit / 32bit = 4 elements per thread at each step
  const uint innerRowA = threadIdx.x / (BK / 4);
  const uint innerColA = threadIdx.x % (BK / 4);
  constexpr uint rowStrideA = (NUM_THREADS * 4) / BK;
  const uint innerRowB = threadIdx.x / (BN / 4);
  const uint innerColB = threadIdx.x % (BN / 4);
  constexpr uint rowStrideB = NUM_THREADS / (BN / 4);

  // allocate thread-local cache for results in registerfile
  float threadResults[WMITER * TM * WNITER * TN] = {0.0};
  // we cache into registers on the warptile level
  float regM[WMITER * TM] = {0.0};
  float regN[WNITER * TN] = {0.0};

  // outer-most loop over block tiles
  for (uint bkIdx = 0; bkIdx < K; bkIdx += BK) {
    loadFromGmem<BM, BN, BK, rowStrideA, rowStrideB>(
        N, K, A, B, As, Bs, innerRowA, innerColA, innerRowB, innerColB);
    __syncthreads();
    processFromSmem<BM, BN, BK, WM, WN, WMITER, WNITER, WSUBM, WSUBN, TM,
                        TN>(regM, regN, threadResults, As, Bs, warpRow, warpCol,
                            threadRowInWarp, threadColInWarp);
    A += BK;     // move BK columns to right
    B += BK * N; // move BK rows down
    __syncthreads();
  }
  // write out the results
  for (uint wSubRowIdx = 0; wSubRowIdx < WMITER; ++wSubRowIdx) {
    for (uint wSubColIdx = 0; wSubColIdx < WNITER; ++wSubColIdx) {
      // move C pointer to current warp subtile
      float *C_interim = C + (wSubRowIdx * WSUBM) * N + wSubColIdx * WSUBN;
      for (uint resIdxM = 0; resIdxM < TM; resIdxM += 1) {
        for (uint resIdxN = 0; resIdxN < TN; resIdxN += 4) {
          // load C vector into registers
          float4 tmp = reinterpret_cast<float4 *>(
              &C_interim[(threadRowInWarp * TM + resIdxM) * N +
                        threadColInWarp * TN + resIdxN])[0];
          // perform GEMM update in reg
          const int i = (wSubRowIdx * TM + resIdxM) * (WNITER * TN) +
                        wSubColIdx * TN + resIdxN;
          tmp.x = threadResults[i + 0];
          tmp.y = threadResults[i + 1];
          tmp.z = threadResults[i + 2];
          tmp.w = threadResults[i + 3];
          // write back
          reinterpret_cast<float4 *>(
              &C_interim[(threadRowInWarp * TM + resIdxM) * N +
                        threadColInWarp * TN + resIdxN])[0] = tmp;
        }
      }
    }
  }
}

template<typename T>
__host__ void copyFromHostToDevice(T* h_a, T* h_b, T* d_a, T* d_b, size_t M, size_t N , size_t K) {
  size_t a_bytes = sizeof(T) * M * K;
  size_t b_bytes = sizeof(T) * K * N;
  cudaError_t err = cudaMemcpy(d_a, h_a, a_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_a to d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaMemcpy(d_b, h_b, b_bytes, cudaMemcpyHostToDevice);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy h_b to d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T, const uint BM, const uint BN, const uint BK, const uint TM, const uint TN, const uint WM, const uint WN, const uint WNITER>
__host__ void executeKernel(T* d_a, T* d_b, T* d_c, size_t M, size_t N, size_t K) {
  dim3 block(128);
  dim3 grid((M + BM - 1) / BM, (N + BN - 1) / BN, 1);
  gemm_kernel<T, BM, BN, BK, TM, TN, WM, WN, WNITER><<<grid, block>>>(d_a, d_b, d_c, M, N, K);
  cudaDeviceSynchronize();
}

template<typename T>
__host__ void copyFromDeviceToHost(T* d_c, T* h_c, size_t M, size_t N) {
  size_t c_bytes = sizeof(T) * M * N;
  cudaError_t err = cudaMemcpy(h_c, d_c, c_bytes, cudaMemcpyDeviceToHost);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to copy from d_c to h_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

template<typename T>
__host__ void deallocateMemory(T* d_a, T* d_b, T* d_c) {
  cudaError_t err = cudaFree(d_a);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_a (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_b);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_b (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
  err = cudaFree(d_c);
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to deallocate d_c (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ void cleanUpDevice() {
  cudaError_t err = cudaDeviceReset();
  if (err != cudaSuccess) {
    fprintf(stderr, "Failed to clean up device (error code: %s)", cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

__host__ std::tuple<int, int, int> parseCmdLineArgs(int argc, char *argv[]) {
  int M = 1024;
  int N = 1024;
  int K = 1024;

  for (int i = 1; i < argc; i++){
    std::string option(argv[i]);
    std::string value(argv[i+1]);
    i++;
    if (option.compare("-m") == 0) {
      M = std::stoi(value);
    }
    else if (option.compare("-n") == 0) {
      N = std::stoi(value);
    }
    else if (option.compare("-k") == 0) {
      K = std::stoi(value);
    }
  }
  return {M, N, K};
}

int main(int argc, char *argv[]) {
std::tuple<int, int, int>parsedCmdLineArgsTuple = parseCmdLineArgs(argc, argv);
  int M = std::get<0>(parsedCmdLineArgsTuple);
  int N = std::get<1>(parsedCmdLineArgsTuple);
  int K = std::get<2>(parsedCmdLineArgsTuple);

  float* h_a = (float*)malloc(M * K * sizeof(float));
  float* h_b = (float*)malloc(K * N * sizeof(float));
  float* h_c = (float*)malloc(M * N * sizeof(float));

  // initialize
  for (size_t i = 0; i < M; i++) {
    for (size_t j = 0; j < K; j++) {
      h_a[i * K + j] = rand() % 10;
    }
  }

  for (size_t i = 0; i < K; i++) {
    for (size_t j = 0; j < N; j++) {
      h_b[i * N + j] = rand() % 10;
    }
  }

  // allocate memory on device side
  float *d_a, *d_b, *d_c;
  cudaMalloc((float **)&d_a, M * K * sizeof(float));
  cudaMalloc((float **)&d_b, K * N * sizeof(float));
  cudaMalloc((float **)&d_c, M * N * sizeof(float));

  copyFromHostToDevice<float>(h_a, h_b, d_a, d_b, M, N, K);

  //cudaEvent_t start, stop;
  //float time;
  //cudaEventCreate(&start);
  //cudaEventCreate(&stop);
//
  //cudaEventRecord( start, 0 );

  // T, BM, BN, BK, TM, TN, WM, WN, WNITER
  executeKernel<float, 128, 128, 16, 8, 4, 64, 64, 4>(d_a, d_b, d_c, M, N, K);

  //cudaEventRecord( stop, 0 );
  //cudaEventSynchronize( stop );
//
  //cudaEventElapsedTime( &time, start, stop );
  //printf("Time taken for GEMM: %f ms\n", time);
  //cudaEventDestroy( start );
  //cudaEventDestroy( stop );
//
  //std::cout << "Performance: " << 2LL*M*N*K/(time * 1e-3 * 1e9) << " GFLOP/s\n";

  copyFromDeviceToHost<float>(d_c, h_c, M, N);
  //verifyResult<float>(h_a, h_b, h_c, M, N, K);
  deallocateMemory<float>(d_a, d_b, d_c);
  cleanUpDevice();
  return 0;
}

Writing /content/src/warp-tiled-cc-main.cu


In [ ]:
%%bash
nvcc -o /content/build/warp_tiled_cc_out /content/src/warp-tiled-cc-main.cu -arch=sm_75 -Xptxas=-v
cd /content/build
./warp_tiled_cc_out -m 1024 -n 1024 -k 1024

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z11gemm_kernelIfLm128ELm128ELm16ELm8ELm4ELm64ELm64ELm4EEvPT_S1_S1_mmm' for 'sm_75'
ptxas info    : Function properties for _Z11gemm_kernelIfLm128ELm128ELm16ELm8ELm4ELm64ELm64ELm4EEvPT_S1_S1_mmm
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 223 registers, 16384 bytes smem, 400 bytes cmem[0]


In [ ]:
%%bash
cd /content/build
ncu -f --set full --import-source on -o  /content/warp_tiled_cc warp_tiled_cc_out -m 1024 -n 1024 -k 1024
ncu --print-details all --print-rule-details --set full ./warp_tiled_cc_out -m 1024 -n 1024 -k 1024 > /content/report/warp_tiled_cc.report

==PROF== Connected to process 9580 (/content/build/warp_tiled_cc_out)
==PROF== Profiling "gemm_kernel" - 0: 0%....50%....100% - 31 passes
==PROF== Disconnected from process 9580
==WARNING== No source files were imported. Check that the target application was compiled with -lineinfo.
==PROF== Report: /content/warp_tiled_cc.ncu-rep


# CuBLAS - 2893.74 GFLOP/s

In [ ]:
%%writefile /content/src/cublas-main.cu

#include <cublas_v2.h>
#include <cuda_runtime.h>
#include <iostream>
#include <chrono>
#include <tuple>

void checkCudaError(cudaError_t status, const char* msg) {
    if (status != cudaSuccess) {
        std::cerr << msg << " Error: " << cudaGetErrorString(status) << std::endl;
        exit(EXIT_FAILURE);
    }
}

void checkCublasError(cublasStatus_t status, const char* msg) {
    if (status != CUBLAS_STATUS_SUCCESS) {
        std::cerr << msg << " Error: " << status << std::endl;
        exit(EXIT_FAILURE);
    }
}

__host__ std::tuple<int, int, int> parseCmdLineArgs(int argc, char *argv[]) {
  int M = 1024;
  int N = 1024;
  int K = 1024;

  for (int i = 1; i < argc; i++){
    std::string option(argv[i]);
    std::string value(argv[i+1]);
    i++;
    if (option.compare("-m") == 0) {
      M = std::stoi(value);
    }
    else if (option.compare("-n") == 0) {
      N = std::stoi(value);
    }
    else if (option.compare("-k") == 0) {
      K = std::stoi(value);
    }
  }
  return {M, N, K};
}

int main(int argc, char *argv[]) {
    std::tuple<int, int, int>parsedCmdLineArgsTuple = parseCmdLineArgs(argc, argv);
    int M = std::get<0>(parsedCmdLineArgsTuple);
    int N = std::get<1>(parsedCmdLineArgsTuple);
    int K = std::get<2>(parsedCmdLineArgsTuple);

    const float alpha = 1.0f, beta = 0.0f;

    // Allocate host memory
    float *h_A, *h_B, *h_C;
    h_A = new float[M * K];
    h_B = new float[K * N];
    h_C = new float[M * N];

      // initialize
    for (size_t i = 0; i < M; i++) {
      for (size_t j = 0; j < K; j++) {
        h_A[i * K + j] = rand() % 10;
      }
    }

    for (size_t i = 0; i < K; i++) {
      for (size_t j = 0; j < N; j++) {
        h_B[i * N + j] = rand() % 10;
      }
    }

    for (size_t i = 0; i < M; i++) {
      for (size_t j = 0; j < N; j++) {
        h_C[i * N + j] = 0;
      }
    }

    // Allocate device memory
    float *d_A, *d_B, *d_C;
    checkCudaError(cudaMalloc(&d_A, M * K * sizeof(float)), "CUDA malloc failed for d_A");
    checkCudaError(cudaMalloc(&d_B, K * N * sizeof(float)), "CUDA malloc failed for d_B");
    checkCudaError(cudaMalloc(&d_C, M * N * sizeof(float)), "CUDA malloc failed for d_C");

    // Copy data to device
    checkCudaError(cudaMemcpy(d_A, h_A, M * K * sizeof(float), cudaMemcpyHostToDevice), "Memcpy to d_A failed");
    checkCudaError(cudaMemcpy(d_B, h_B, K * N * sizeof(float), cudaMemcpyHostToDevice), "Memcpy to d_B failed");

    // Create cuBLAS handle
    cublasHandle_t handle;
    checkCublasError(cublasCreate(&handle), "cuBLAS initialization failed");

    // Perform GEMM
    checkCublasError(cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N,
                                 M, N, K, &alpha, d_A, M, d_B, K, &beta, d_C, M),
                     "cuBLAS Sgemm failed");

    // Cleanup
    delete[] h_A;
    delete[] h_B;
    delete[] h_C;
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cublasDestroy(handle);

    return 0;
}

Overwriting /content/src/cublas-main.cu


In [ ]:
%%bash
nvcc -o /content/build/cublas_out /content/src/cublas-main.cu -lcublas -arch=sm_75 -Xptxas=-v
cd /content/build
./cublas_out -m 1024 -n 1024 -k 1024

ptxas info    : 0 bytes gmem


In [ ]:
%%bash
cd /content/build
ncu -f --set full --import-source on -o  /content/cublas cublas_out -m 1024 -n 1024 -k 1024
ncu --print-details all --print-rule-details --set full ./cublas_out -m 1024 -n 1024 -k 1024> /content/report/cublas.report

==PROF== Connected to process 6267 (/content/build/cublas_out)
==PROF== Profiling "volta_sgemm_128x64_nn" - 0: 0%.
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
...50%....100% - 31 passes
==PROF== Profiling "volta_sgemm_128x64_nn" - 1: 0%....50%....100% - 31 passes
==PROF== Disconnected from process 6267
==WARNING== No source files were imported. Check that the target application was compiled with -lineinfo.
==PROF== Report: /content/cublas.ncu-rep


# CUTLASS - 2684.27 GFLOP/s

In [ ]:
!git clone https://github.com/NVIDIA/cutlass.git

Cloning into 'cutlass'...
remote: Enumerating objects: 28742, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (410/410), done.
remote: Total 28742 (delta 388), reused 176 (delta 176), pack-reused 28156 (from 3)
Receiving objects: 100% (28742/28742), 45.96 MiB | 24.04 MiB/s, done.
Resolving deltas: 100% (21226/21226), done.


In [ ]:
%%writefile /content/src/cutlass-main.cu

#include <iostream>
#include <cutlass/numeric_types.h>
#include <cutlass/gemm/device/gemm.h>
#include <tuple>

#include <cutlass/util/host_tensor.h>

__host__ std::tuple<int, int, int> parseCmdLineArgs(int argc, char *argv[]) {
  int M = 1024;
  int N = 1024;
  int K = 1024;

  for (int i = 1; i < argc; i++){
    std::string option(argv[i]);
    std::string value(argv[i+1]);
    i++;
    if (option.compare("-m") == 0) {
      M = std::stoi(value);
    }
    else if (option.compare("-n") == 0) {
      N = std::stoi(value);
    }
    else if (option.compare("-k") == 0) {
      K = std::stoi(value);
    }
  }
  return {M, N, K};
}

int main(int argc, char *argv[]) {

  // Define the GEMM operation
  using Gemm = cutlass::gemm::device::Gemm<
    float,                           // ElementA
    cutlass::layout::ColumnMajor,              // LayoutA
    float,                           // ElementB
    cutlass::layout::ColumnMajor,              // LayoutB
    float,                           // ElementOutput
    cutlass::layout::ColumnMajor,              // LayoutOutput
    float,                                     // ElementAccumulator
    cutlass::arch::OpClassSimt,            // tag indicating Tensor Cores
    cutlass::arch::Sm75                        // tag indicating target GPU compute architecture
  >;

  Gemm gemm_op;
  cutlass::Status status;

  //
  // Define the problem size
  //
  std::tuple<int, int, int>parsedCmdLineArgsTuple = parseCmdLineArgs(argc, argv);
  int M = std::get<0>(parsedCmdLineArgsTuple);
  int N = std::get<1>(parsedCmdLineArgsTuple);
  int K = std::get<2>(parsedCmdLineArgsTuple);

  float alpha = 1.25f;
  float beta = -1.25f;

  //
  // Allocate device memory
  //

  cutlass::HostTensor<float, cutlass::layout::ColumnMajor> A({M, K});
  cutlass::HostTensor<float, cutlass::layout::ColumnMajor> B({K, N});
  cutlass::HostTensor<float, cutlass::layout::ColumnMajor> C({M, N});

  float const *ptrA = A.device_data();
  float const *ptrB = B.device_data();
  float const *ptrC = C.device_data();
  float       *ptrD = C.device_data();

  int lda = A.device_ref().stride(0);
  int ldb = B.device_ref().stride(0);
  int ldc = C.device_ref().stride(0);
  int ldd = C.device_ref().stride(0);
  //
  // Launch GEMM on the device
  //

  status = gemm_op({
    {M, N, K},
    {ptrA, lda},            // TensorRef to A device tensor
    {ptrB, ldb},            // TensorRef to B device tensor
    {ptrC, ldc},            // TensorRef to C device tensor
    {ptrD, ldd},            // TensorRef to D device tensor - may be the same as C
    {alpha, beta}           // epilogue operation arguments
  });

  if (status != cutlass::Status::kSuccess) {
    std::cout << "GEMM failed\n";
    return -1;
  }

  return 0;
}

Overwriting /content/src/cutlass-main.cu


In [ ]:
%%bash
nvcc -I/content/cutlass/include -I/content/cutlass/tools/util/include /content/src/cutlass-main.cu -o /content/build/cutlass_out --expt-relaxed-constexpr -arch=sm_75 -Xptxas=-v
cd /content/build
./cutlass_out

ptxas info    : 500 bytes gmem
ptxas info    : Compiling entry function '_ZN7cutlass6KernelINS_4gemm6kernel4GemmINS1_11threadblock12MmaPipelinedINS1_9GemmShapeILi128ELi128ELi8EEENS_9transform11threadblock22PredicatedTileIteratorINS_11MatrixShapeILi128ELi8EEEfNS_6layout8RowMajorELi1ENS8_30PitchLinearStripminedThreadMapINS_16PitchLinearShapeILi8ELi128EEELi256ELi1EEELi1ELb0ENSD_9NoPermuteEEENS9_19RegularTileIteratorISC_fNSD_11ColumnMajorELi1ENS8_33TransposePitchLinearThreadMapSimtISI_EELi4EEENSA_INSB_ILi8ELi128EEEfSE_Li0ENSF_INSG_ILi128ELi8EEELi256ELi1EEELi1ELb0ESJ_EENSL_ISQ_fSE_Li0ESS_Li4EEEfSE_NS4_9MmaPolicyINS1_4warp7MmaSimtINS6_ILi32ELi64ELi8EEEfSM_fSE_fSE_NSW_13MmaSimtPolicyINSB_ILi4ELi8EEENSD_19RowMajorInterleavedILi2EEENS6_ILi4ELi4ELi1EEEEELi1ELNS_16ComplexTransformE0ELS15_0EbEENSB_ILi4ELi0EEENSB_ILi0ELi0EEELi1EEENS_21NumericArrayConverterIffLi4ELNS_15FloatRoundStyleE2ENS8_6thread14UnaryTransform8IdentityEEES1F_bEENS_8epilogue11threadblock8EpilogueIS7_S16_Li1ENS1I_22PredicatedTileI

In [ ]:
%%bash
cd /content/build
ncu -f --set full --import-source on -o  /content/cutlass cutlass_out
ncu --print-details all --print-rule-details --set full ./cutlass_out > /content/report/cutlass.report

==PROF== Connected to process 5349 (/content/build/cutlass_out)
==PROF== Profiling "Kernel" - 0: 0%....50%....100% - 31 passes
==PROF== Disconnected from process 5349
==WARNING== No source files were imported. Check that the target application was compiled with -lineinfo.
==PROF== Report: /content/cutlass.ncu-rep


# Summary

In [ ]:
%%bash
cd /content/build
# ncu --metrics gpu__time_duration.sum ./naive_out -m 512 -n 512 -k 512 | grep gpu__time_duration.sum > summary_naive.txt
# ncu --metrics gpu__time_duration.sum ./naive_out -m 1024 -n 1024 -k 1024 | grep gpu__time_duration.sum >> summary_naive.txt
# ncu --metrics gpu__time_duration.sum ./naive_out -m 2048 -n 2048 -k 2048 | grep gpu__time_duration.sum >> summary_naive.txt
# ncu --metrics gpu__time_duration.sum ./naive_out -m 4096 -n 4096 -k 4096 | grep gpu__time_duration.sum >> summary_naive.txt

# ncu --metrics gpu__time_duration.sum ./tiled_out -m 512 -n 512 -k 512 | grep gpu__time_duration.sum > summary_tiled.txt
# ncu --metrics gpu__time_duration.sum ./tiled_out -m 1024 -n 1024 -k 1024 | grep gpu__time_duration.sum >> summary_tiled.txt
# ncu --metrics gpu__time_duration.sum ./tiled_out -m 2048 -n 2048 -k 2048 | grep gpu__time_duration.sum >> summary_tiled.txt
# ncu --metrics gpu__time_duration.sum ./tiled_out -m 4096 -n 4096 -k 4096 | grep gpu__time_duration.sum >> summary_tiled.txt

# ncu --metrics gpu__time_duration.sum ./tiled_ilp_2D_out -m 512 -n 512 -k 512 | grep gpu__time_duration.sum > summary_tiled_2d.txt
# ncu --metrics gpu__time_duration.sum ./tiled_ilp_2D_out -m 1024 -n 1024 -k 1024 | grep gpu__time_duration.sum >> summary_tiled_2d.txt
# ncu --metrics gpu__time_duration.sum ./tiled_ilp_2D_out -m 2048 -n 2048 -k 2048 | grep gpu__time_duration.sum >> summary_tiled_2d.txt
# ncu --metrics gpu__time_duration.sum ./tiled_ilp_2D_out -m 4096 -n 4096 -k 4096 | grep gpu__time_duration.sum >> summary_tiled_2d.txt

# ncu --metrics gpu__time_duration.sum ./warp_tiled_cc_out -m 512 -n 512 -k 512 | grep gpu__time_duration.sum > summary_warp_tiled.txt
# ncu --metrics gpu__time_duration.sum ./warp_tiled_cc_out -m 1024 -n 1024 -k 1024 | grep gpu__time_duration.sum >> summary_warp_tiled.txt
# ncu --metrics gpu__time_duration.sum ./warp_tiled_cc_out -m 2048 -n 2048 -k 2048 | grep gpu__time_duration.sum >> summary_warp_tiled.txt
# ncu --metrics gpu__time_duration.sum ./warp_tiled_cc_out -m 4096 -n 4096 -k 4096 | grep gpu__time_duration.sum >> summary_warp_tiled.txt

ncu --metrics gpu__time_duration.sum ./cublas_out -m 512 -n 512 -k 512 | grep gpu__time_duration.sum > summary_cublas.txt
ncu --metrics gpu__time_duration.sum ./cublas_out -m 1024 -n 1024 -k 1024 | grep gpu__time_duration.sum >> summary_cublas.txt
ncu --metrics gpu__time_duration.sum ./cublas_out -m 2048 -n 2048 -k 2048 | grep gpu__time_duration.sum >> summary_cublas.txt
ncu --metrics gpu__time_duration.sum ./cublas_out -m 4096 -n 4096 -k 4096 | grep gpu__time_duration.sum >> summary_cublas.txt

# ncu --metrics gpu__time_duration.sum ./cutlass_out -m 512 -n 512 -k 512 | grep gpu__time_duration.sum > summary_cutlass.txt
# ncu --metrics gpu__time_duration.sum ./cutlass_out -m 1024 -n 1024 -k 1024 | grep gpu__time_duration.sum >> summary_cutlass.txt
# ncu --metrics gpu__time_duration.sum ./cutlass_out -m 2048 -n 2048 -k 2048 | grep gpu__time_duration.sum >> summary_cutlass.txt
# ncu --metrics gpu__time_duration.sum ./cutlass_out -m 4096 -n 4096 -k 4096 | grep gpu__time_duration.sum >> summary_cutlass.txt